In [1]:
import cv2
import os
import time, gc
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model, Input
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.layers import Dropout, BatchNormalization, Lambda
from math import ceil
from tqdm.auto import tqdm

# Install EfficientNet
!pip install efficientnet-1.0.0-py3-none-any.whl
import efficientnet.keras as efn

Using TensorFlow backend.


In [2]:
# Constants
HEIGHT = 137
WIDTH = 236
FACTOR = 1.00
HEIGHT_NEW = int(HEIGHT * FACTOR)
WIDTH_NEW = int(WIDTH * FACTOR)
CHANNELS = 3
BATCH_SIZE = 64

# Dir
DIR = 'C:/bengaliai-cv19'

## Image Preprocessing

In [3]:
# Image Size Summary
print(HEIGHT_NEW)
print(WIDTH_NEW)

# Image Prep
def resize_image(img, WIDTH_NEW, HEIGHT_NEW):
    # Invert
    img = 255 - img

    # Normalize
    img = (img * (255.0 / img.max())).astype(np.uint8)

    # Reshape
    img = img.reshape(HEIGHT, WIDTH)
    image_resized = cv2.resize(img, (WIDTH_NEW, HEIGHT_NEW), interpolation = cv2.INTER_AREA)
    
    return image_resized.reshape(-1)   

137
236


## Create Model

In [4]:
# Generalized mean pool - GeM
gm_exp = tf.Variable(3.0, dtype = tf.float32)
def generalized_mean_pool_2d(X):
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool

In [5]:
def create_model(input_shape):
    # Input Layer
    input = Input(shape = input_shape)
    
    # Create and Compile Model and show Summary
    x_model = efn.EfficientNetB3(weights = None, include_top = False, input_tensor = input, pooling = None, classes = None)
    
    # Root
    lambda_layer1 = Lambda(generalized_mean_pool_2d)
    lambda_layer1.trainable_weights.extend([gm_exp])
    x1 = lambda_layer1(x_model.output)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.3)(x1)
    x1 = Dense(512, activation = 'relu')(x1)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.3)(x1)
    
    # Vowel
    lambda_layer2 = Lambda(generalized_mean_pool_2d)
    lambda_layer2.trainable_weights.extend([gm_exp])
    x2 = lambda_layer2(x_model.output)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    x2 = Dense(512, activation = 'relu')(x2)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    
    # Consonant
    lambda_layer3 = Lambda(generalized_mean_pool_2d)
    lambda_layer3.trainable_weights.extend([gm_exp])
    x3 = lambda_layer3(x_model.output)
    x3 = BatchNormalization()(x3)
    x3 = Dropout(0.3)(x3)
    x3 = Dense(512, activation = 'relu')(x3)
    x3 = BatchNormalization()(x3)
    x3 = Dropout(0.3)(x3)

    # multi output
    grapheme_root = Dense(168, activation = 'softmax', name = 'root')(x1)
    vowel_diacritic = Dense(11, activation = 'softmax', name = 'vowel')(x2)
    consonant_diacritic = Dense(7, activation = 'softmax', name = 'consonant')(x3)

    # model
    model = Model(inputs = x_model.input, outputs = [grapheme_root, vowel_diacritic, consonant_diacritic])

    return model

## Data Generator

In [6]:
class TestDataGenerator(keras.utils.Sequence):
    def __init__(self, X, batch_size = 16, img_size = (512, 512, 3), *args, **kwargs):
        self.X = X
        self.indices = np.arange(len(self.X))
        self.batch_size = batch_size
        self.img_size = img_size
                    
    def __len__(self):
        return int(ceil(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indices)
        return X
    
    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        
        for i, index in enumerate(indices):
            image = self.X[index]
            image = np.stack((image,)*CHANNELS, axis=-1)
            image = image.reshape(-1, HEIGHT_NEW, WIDTH_NEW, CHANNELS)
            
            X[i,] = image
        
        return X 

## Predict and Submission

In [7]:
# Create Model
model1 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
model2 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
model3 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
model4 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))

# Load Model Weights
model1.load_weights('./model_weights/Train2_model_61.h5')
model2.load_weights('./model_weights/Train2_model_62.h5')
model3.load_weights('./model_weights/Train2_model_63.h5')
model4.load_weights('./model_weights/Train2_model_64.h5')

# Create Submission File
tgt_cols = ['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']

# Create Predictions
row_ids, targets = [], []

# Loop through Test Parquet files (X)
for i in tqdm(range(0, 4)):
    # Test Files Placeholder
    test_files = []

    # Read Parquet file
    df = pd.read_parquet(os.path.join(DIR, 'test_image_data_'+str(i)+'.parquet'))
    # Get Image Id values
    image_ids = df['image_id'].values 
    # Drop Image_id column
    df = df.drop(['image_id'], axis = 1)

    # Loop over rows in Dataframe and generate images 
    X = []
    for image_id, index in zip(image_ids, range(df.shape[0])):
        test_files.append(image_id)
        X.append(resize_image(df.loc[df.index[index]].values, WIDTH_NEW, HEIGHT_NEW))

    # Data_Generator
    data_generator_test = TestDataGenerator(X, batch_size = BATCH_SIZE, img_size = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))
        
    # Predict
    preds1 = model1.predict_generator(data_generator_test, verbose = 1)
    preds2 = model2.predict_generator(data_generator_test, verbose = 1)
    preds3 = model3.predict_generator(data_generator_test, verbose = 1)
    preds4 = model4.predict_generator(data_generator_test, verbose = 1)
    
    # Loop over Preds    
    for i, image_id in zip(range(len(test_files)), test_files):
        
        for subi, col in zip(range(len(preds1)), tgt_cols):
            sub_preds1 = preds1[subi]
            sub_preds2 = preds2[subi]
            sub_preds3 = preds3[subi]
            sub_preds4 = preds4[subi]
            
            # Set Prediction
            row_ids.append(str(image_id)+'_'+col)
            sub_pred_value = np.argmax((sub_preds1[i] + sub_preds2[i] + sub_preds3[i] + sub_preds4[i]) / 4)
            targets.append(sub_pred_value)
    
    # Cleanup
    del df
    gc.collect()

1/1 [==============================] - 8s 8s/step



In [8]:
# Create and Save Submission File
submit_df = pd.DataFrame({'row_id':row_ids,'target':targets}, columns = ['row_id','target'])
submit_df.to_csv('submission.csv', index = False)
print(submit_df.head(40))

                         row_id  target
0          Test_0_grapheme_root       3
1        Test_0_vowel_diacritic       0
2    Test_0_consonant_diacritic       0
3          Test_1_grapheme_root      93
4        Test_1_vowel_diacritic       2
5    Test_1_consonant_diacritic       0
6          Test_2_grapheme_root      19
7        Test_2_vowel_diacritic       0
8    Test_2_consonant_diacritic       0
9          Test_3_grapheme_root     115
10       Test_3_vowel_diacritic       0
11   Test_3_consonant_diacritic       0
12         Test_4_grapheme_root      55
13       Test_4_vowel_diacritic       4
14   Test_4_consonant_diacritic       0
15         Test_5_grapheme_root     115
16       Test_5_vowel_diacritic       2
17   Test_5_consonant_diacritic       0
18         Test_6_grapheme_root     147
19       Test_6_vowel_diacritic       9
20   Test_6_consonant_diacritic       5
21         Test_7_grapheme_root     137
22       Test_7_vowel_diacritic       7
23   Test_7_consonant_diacritic       0
